In [1]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

load_dotenv()

True

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [3]:
tools = [TavilySearchResults(max_results=1)]

In [4]:
model = ChatGroq(temperature=0,model="llama3-70b-8192", streaming=False)

In [5]:
model = model.bind_tools(tools)

In [6]:
tool_node = ToolNode(tools)

In [7]:
def add_messages(left: list, right: list):
    """Añadir sin sobrescribir."""
    return left + right

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [9]:
# Definir la función que determina si continuar o no
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # Si el LLM hace una llamada a una herramienta, entonces enrutamos al nodo "tools"
    if last_message.tool_calls:
        return "tools"
    # De lo contrario, nos detenemos (respondemos al usuario)
    return "__end__"

In [10]:
# Definir la función que llama al modelo
def call_model(state: AgentState):
    messages = state['messages']
    
    response = model.invoke(messages)
 
    # Devolvemos una lista, porque esto se agregará a la lista existente
    return {"messages": [response]}

## Definir el Grafo

In [11]:
# Definir un nuevo grafo
workflow = StateGraph(AgentState)


In [12]:
# Definir los dos nodos entre los que ciclará
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [13]:
# Establecer el punto de entrada como `agent`
workflow.set_entry_point("agent")

In [14]:
# Agregar un borde condicional
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

In [15]:
# Agregar un borde normal de `tools` a `agent`
workflow.add_edge('tools', 'agent')

In [16]:
# Finalmente, ¡compilamos!
app = workflow.compile()

In [17]:
mensaje = HumanMessage(content="¿Cuál es el clima en San Francisco? Contesta en español")

In [18]:
inputs = {"messages": [mensaje]}
inputs


{'messages': [HumanMessage(content='¿Cuál es el clima en San Francisco? Contesta en español')]}

In [19]:
app.invoke(inputs)

{'messages': [HumanMessage(content='¿Cuál es el clima en San Francisco? Contesta en español'),
  AIMessage(content='El clima en San Francisco es conocido por ser fresco y húmedo durante todo el año. La temperatura promedio anual es de alrededor de 14°C (57°F). La ciudad experimenta un clima mediterráneo, con veranos secos y frescos, e inviernos húmedos y frescos.\n\nEn verano, las temperaturas máximas suelen rondar los 22°C (72°F), mientras que en invierno, las temperaturas mínimas pueden llegar a ser de alrededor de 8°C (46°F). La niebla es común en San Francisco, especialmente en los meses de verano, lo que puede hacer que la temperatura se sienta más fresca de lo que realmente es.\n\nEn cuanto a la precipitación, San Francisco recibe la mayoría de su lluvia durante los meses de invierno, con un promedio anual de alrededor de 600 mm (24 pulgadas).', response_metadata={'token_usage': {'completion_time': 0.679, 'completion_tokens': 211, 'prompt_time': 1.342, 'prompt_tokens': 927, 'queu